<center><img src='https://github.com/MateusPincho/Projeto-Filtros-Ativos/blob/master/docs/logo.png?raw=true'/></center>

<h1 align = 'center'>Trabalho de Circuitos Elétricos II<h1/>
<h2 align = 'center'>Modelagem de Sistema Elétrico de Potência<h2/>

Este projeto tem como objetivo o estudo referente ao dimensionamento de um Filtro Ativo. O projeto está salvo e disponível para consulta em nosso [repositório](https://github.com/MateusPincho/Simulacao-SEP/tree/master).

Projetistas:

- André Vitor Cadena de Oliveira
- Fábio Augusto Almeida Marçal
- Gabriel Nazário Gonçalves
- Lorenzo Carrera de Oliveira
- Mateus Pincho de Oliveira

---
 

### Linha de Transmissão

Segundo a teoria de quadripolos, podemos modelar os componentes do sistema elétrico de potência da seguinte forma: 

<p align = "center">
    <img src = '../docs/linha.jpeg'>
</p>

Linhas de transmissão possuem as suas características distríbuidas por toda linha, sendo a impedância $Z = R + jX_L$ e sua admitância $Y = jX_C$. Para linhas de transmissão entre 80 a 250 quilômetros, podemos concentrar os seus parâmetros numa equação matricial. A tensão e corrente no começo e final da linha de transmissão podem ser determinadas como:

$$ \begin{bmatrix}
V_1\\ 
I_1
\end{bmatrix} = \begin{bmatrix}
1 + \frac{ZY}{2} & Z\\ 
Y+\frac{ZY^{2}}{4} & 1 + \frac{ZY}{2}
\end{bmatrix} \cdot \begin{bmatrix}
V_2\\ 
-I_2
\end{bmatrix} $$

---

### Transformador

<p align = "center">
    <img src = '../docs/transformador.jpeg'>
</p>

Um transformador de um sistema elétrico de potência pode ser modelador como mostrado na figura anterior. A relação de tensão e corrente nos seus terminais pode ser escrita como: 

$$
\begin{bmatrix}
V_1\\ 
I_1
\end{bmatrix} = \begin{bmatrix}
\frac{1}{n}(1+YZ_1) &n(Z_1+Z_2+YZ_1Z_2) \\ 
\frac{1}{n}Y & n(1+YZ_2)
\end{bmatrix}\cdot \begin{bmatrix}
V_2\\ 
-I_2
\end{bmatrix}
$$

Sendo: 

$$Z_1 = R_1+jX_{L_1}$$
$$Z_2 = R_2+jX_{L_2}$$

$$Y = \frac{R_m + jX_m}{R_m \cdot jX_m}$$

---

### Carga em Série

A tensão e corrente nos terminais de uma carga colocada em série com o circuito é dada por:

$$\begin{bmatrix}
V_1\\ 
I_1
\end{bmatrix} = \begin{bmatrix}
1 &Z \\ 
0 & 1
\end{bmatrix}\cdot \begin{bmatrix}
V_2\\ 
-I_2
\end{bmatrix} $$

---

### Carga em Paralelo

A tensão e corrente nos terminais de uma carga colocada em paralelo com o circuito é dada por:

$$ \begin{bmatrix}
V_1\\ 
I_1
\end{bmatrix} = \begin{bmatrix}
1 &0 \\ 
Y & 1
\end{bmatrix}\cdot \begin{bmatrix}
V_2\\ 
-I_2
\end{bmatrix}$$

---

In [2]:
# Importando os módulos utilizados...
import numpy as np

In [3]:
# Classes para Quadripolos de Sistemas de Transmissão

class Quadripolo:
    def __init__(self, T=np.zeros((2,2))):
        self.T = T  
        self.A = self.T[0][0]
        self.B = self.T[0][1]
        self.C = self.T[1][0]
        self.D = self.T[1][1]

    @staticmethod
    def paralelo(QDP1, QDP2):
        A = (QDP1.A*QDP2.B+QDP2.A*QDP1.B)/(QDP1.B+QDP2.B) 
        B = (QDP1.B*QDP2.B)/(QDP1.B+QDP2.B)             
        C = QDP1.C+QDP2.C+(QDP1.A-QDP2.A)*(QDP2.D-QDP1.D)/(QDP1.B+QDP2.B)
        D = (QDP2.B*QDP1.D+QDP1.B*QDP2.D)/(QDP1.B+QDP2.B)

        T = np.array([[A, B], 
                      [C, D]])
        
        return Quadripolo(T)

class LinhaDeTransmissao(Quadripolo):
    def __init__(self, w, rho_R, rho_L, rho_C, km):

        R, L, C = km * rho_R, km * rho_L, km * rho_C
        Z = complex(R, w*L)
        Y = complex(0, w*C)
        T = np.array([[1+Z*Y/2, Z], 
                      [Y+Z*(Y**2)/4, 1+Z*Y/2]])
        
        super().__init__(T)
        
class Transformador(Quadripolo):
    def __init__(self, n, Z1, Z2, Ym):
        T = np.array([[(1/n)*(1+Ym*Z1), n*(Z1+Z2+Ym*Z1*Z2)], 
                      [(1/n)*Ym, n*(1+Ym*Z2)]])
        
        super().__init__(T)
        
class CargaEmSerie(Quadripolo):
    def __init__(self, Z):
        T = np.array([[1, Z],
                      [0, 1]])  
        
        super().__init__(T)

class CargaEmDerivacao(Quadripolo):
    def __init__(self, Z):
        T = np.array([[1, 0],
                      [1/Z, 1]])  
        
        super().__init__(T)

In [4]:
# Classe para um Sistema Elétrico de Potência

class SEP: 
    def __init__(self, f):
        self.f = f            # Frequência de operação do sistema em Hz
        self.w = 2*np.pi*f    # Frequência de operação do sistema em rad/s
        self.STC = [] # Sequência de Transmissão em Cascata (geração -> distribuição)

    # Retorna o quadripolo total que representa o Sistema Elétrico de Potência
    def QDP(self):
        T = self.STC[0]

        for t in self.STC[1:]:
            T = T @ t

        return Quadripolo(T)

    # Adiciona um quadripolo em cascata com o último quadripolo adicionado
    # Se não há nenhum quadripolo adicionado, adiciona em cascata com a fonte geradora
    def cascata(self, QDP): 
        self.STC.append(QDP.T)

## Questão 1

Considerando o sistema elétrico abaixo operando a $60 \text{ Hz}$:

<p align = "center">
    <img src = '../docs/sistema-eletrico.jpeg'>
</p>

Modele este sistema criando as matrizes de transferência $[T]$ para cada elemento a partir dos dados de entrada do modelo. Caso dois elementos estejam conectados em série, deve-se realizar a conexão em cascata, se não, deve ser feita a conexão em paralelo.

Para realizar a modelagem, considere os seguintes parâmetros: 
- Impedância em série de Thévenin: $R_f = 4\Omega$ e $X_f = 0,38\Omega$
- Parâmetros distribuídos da linha de transmissão: $R = 0,17 \Omega\text{/km}$, $L = 2,18 \text{ mH/km}$
- Considere que ${LT}_1, {LT}_2$ e ${LT}_3$ possuem $80 \text{ km}$ e que ${LT}_4$ possui $120 \text{ km}$
- Considere os seguintes parâmetros para os Transformadores:
  - $R_1 = 7,6 \text{ m}\Omega$
  - $X_1 = 3,8 \text{ m}\Omega$
  - $R_2 = 33,0 \text{ m}\Omega$
  - $X_2 = 0,85 \text{ m}\Omega$
- Para as cargas, considere:
  - $R_1 = 8000\Omega$ e $L_1 = 41\text{H}$
  - $R_2 = 1350,55\Omega$ e $L_2 = 7,83\text{H}$
  - $R_3 = 649\Omega$ e $L_3 = 3,2\text{H}$

---

In [ ]:
# Projetando o SEP requerido

_SEP = SEP(60) # SEP operando a 60 Hz

# Impedância de Thévenin da Fonte
Rf, Xf = 4, 0.38j
Zf = Rf + Xf 
_SEP.cascata(CargaEmSerie(Zf))

# Parâmetros das impedâncias de entrada, saída dos transformadores

Rt1, Xt1 = 7.6e-3, 3.8e-3j
Zt1 = Rt1 + Xt1 # Impedância de entrada

Rt2, Xt2 = 33.9e-3, 0.85e-3j
Zt2 = Rt2 + Xt2 # Impedância de saída

# Transformador 1

Rm1, Xm1 = 4.32e3, 5.05e3j 
Ym1 = 1/Rm1 + 1/Xm1 # Admitância interna do transformador 1

_SEP.cascata(Transformador(500/69, Zt1, Zt2, Ym1))

# Linhas em Paralelo 1 e 2

rho_R = 0.172     # Ohm/km
rho_L = 2.18e-3   # Henry/km
rho_C = 0.0136e-6 # Faraday/km

LT1 = LinhaDeTransmissao(_SEP.w, rho_R, rho_L, rho_C, 80)
LT2 = LinhaDeTransmissao(_SEP.w, rho_R, rho_L, rho_C, 80)

_SEP.cascata(Quadripolo.paralelo(LT1, LT2))

# Carga em Derivação 1

R1, L1 = 8000, 41 
Z1 = complex(R1,(_SEP.w*L1)) 

_SEP.cascata(CargaEmDerivacao(Z1))

# Linha De Transmissão 3

_SEP.cascata(LinhaDeTransmissao(_SEP.w, rho_R, rho_L, rho_C, 80))

# Transformador 2

Rm2, Xm2 = 4.32e5, 5.05e5j 
Ym2 = 1/Rm2 + 1/Xm2 # Admitância interna do transformador 2

_SEP.cascata(Transformador(230/500, Zt1, Zt2, Ym2))

# Carga em Derivação 2

R2, L2 = 1350.55, 7.83 
Z2 = complex(R2,(_SEP.w*L2)) 

_SEP.cascata(CargaEmDerivacao(Z2))

# Linha De Transmissão 4

_SEP.cascata(LinhaDeTransmissao(_SEP.w, rho_R, rho_L, rho_C, 120))

# Transformador 3

Rm3, Xm3 = 4.02e5, 6.07e5j 
Ym3 = 1/Rm3 + 1/Xm3 # Admitância interna do transformador 3

_SEP.cascata(Transformador(69/230, Zt1, Zt2, Ym3))

# Carga em Derivação 3

R3, L3 = 649, 3.2 
Z3 = complex(R3,(_SEP.w*L3))

_SEP.cascata(CargaEmDerivacao(Z3))

# Saída 

Vs = 69e3
Is = Vs / Z3

S = np.array([[Vs], [-Is]])

### Item 1

Obtenha a tensão fasorial de saída $V_{ac}$ e a corrente fasorial no gerador necessários para que a tensão na carga $Z_3$ seja $69\text{kV}_{RMS}$

---

In [6]:
# Entrada 

E = _SEP.QDP().T @ S

print(E)

[[64065.80167238+18903.7160218j ]
 [  185.93800014 +4255.53102003j]]


### Item 2

Obtenha a tensão e a corrente fasoriais nas impedâncias $Z_1$ e $Z_2$

---

### Item 3

Determine o ajuste do TAP $(n)$ dos transformadores $T_1$, $T_2$ e $T_3$ para que as tensões nas cargas sejam, respectivamente, $500\text{kV}$, $230\text{kV}$ e $69\text{kV}$.

---

## Simulação do Sistema em ambiente virtual

Utilizando o modelo apresentado anteriormente para os elementos do sistema elétrico de potência, utilizou-se o software *LTspice* para simular o sistema e comparar os resultados encontrados na modelagem em *Python*.

<p align = "center">
    <img src = '../docs/sistema-lt.jpeg'>
</p>

---